[View in Colaboratory](https://colab.research.google.com/github/KalashRastogi/DL-text/blob/master/Sarcasm.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpnb4lzf35/pubring.gpg' created
gpg: /tmp/tmpnb4lzf35/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
!pwd

/content


In [0]:
import os
os.chdir("drive/SarcasmApp")

In [5]:
!pwd

/content/drive/SarcasmApp


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
!ls

dl_text		  s2.py		 sarcasm.py	 sarcasm.zip
glove.6B.50d.txt  Sarcasm.ipynb  sarcasm_v2.csv


In [7]:
from dl_text import dl

Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np
from keras.layers import Input,Dense,Dropout,merge,Conv1D,Lambda,Flatten,MaxPooling1D
from keras.models import Model
import keras

In [0]:
def model_cnn2(dimx, dimy, embedding_matrix):
    inpx = Input(shape=(dimx,),dtype='int32',name='inpx')   
    embedx = dl.word2vec_embedding_layer(embedding_matrix)(inpx)
    inpy = Input(shape=(dimy,),dtype='int32',name='inpy')   
    embedy = dl.word2vec_embedding_layer(embedding_matrix)(inpy)
    
    sent_l = Conv1D(nb_filter=3,filter_length=2,activation='relu')(embedx)
    sent_r = Conv1D(nb_filter=3,filter_length=2,activation='relu')(embedy)
    pool_l = MaxPooling1D()(sent_l)
    pool_r = MaxPooling1D()(sent_r)

    combine  = keras.layers.Add()([pool_l,pool_r])
    flat_embed = Flatten()(combine)
    nnet_h = Dense(units=10,activation='sigmoid')(flat_embed)
    nnet_out = Dense(units=2,activation='sigmoid')(nnet_h)
    model = Model([inpx,inpy],nnet_out)
    model.compile(loss='mse',optimizer='adam')
    
    return model

In [10]:
df=pd.read_csv('sarcasm_v2.csv')
df.head()

,Corpus,Label,ID,Quote Text,Response Text
0,GEN,sarc,GEN_sarc_0000,"First off, That's grade A USDA approved Libera...",Therefore you accept that the Republican party...
1,GEN,sarc,GEN_sarc_0001,watch it. Now you're using my lines. Poet has ...,More chattering from the peanut gallery? Haven...
2,GEN,sarc,GEN_sarc_0002,Because it will encourage teens to engage in r...,"Yep, suppressing natural behavior is always th..."
3,GEN,sarc,GEN_sarc_0003,Obviously you missed the point. So sorry the t...,"I guess we all missed your point Justine, what..."
4,GEN,sarc,GEN_sarc_0004,This is pure paranoia. What evidence do you ha...,"Evidence, I dont need no sticking evidence. Th..."


In [11]:
df=df.drop(columns=['Corpus','ID'])
df.head()

,Label,Quote Text,Response Text
0,sarc,"First off, That's grade A USDA approved Libera...",Therefore you accept that the Republican party...
1,sarc,watch it. Now you're using my lines. Poet has ...,More chattering from the peanut gallery? Haven...
2,sarc,Because it will encourage teens to engage in r...,"Yep, suppressing natural behavior is always th..."
3,sarc,Obviously you missed the point. So sorry the t...,"I guess we all missed your point Justine, what..."
4,sarc,This is pure paranoia. What evidence do you ha...,"Evidence, I dont need no sticking evidence. Th..."


In [12]:
df = df.replace({'sarc':1,'notsarc':0})
df.head()

,Label,Quote Text,Response Text
0,1,"First off, That's grade A USDA approved Libera...",Therefore you accept that the Republican party...
1,1,watch it. Now you're using my lines. Poet has ...,More chattering from the peanut gallery? Haven...
2,1,Because it will encourage teens to engage in r...,"Yep, suppressing natural behavior is always th..."
3,1,Obviously you missed the point. So sorry the t...,"I guess we all missed your point Justine, what..."
4,1,This is pure paranoia. What evidence do you ha...,"Evidence, I dont need no sticking evidence. Th..."


In [13]:
label = df['Label'].tolist()
quote = df['Quote Text'].tolist()
response = df['Response Text'].tolist()
print(label[0:5])
print(quote[0:5])
print(response[0:5])

[1, 1, 1, 1, 1]
["First off, That's grade A USDA approved Liberalism in a nutshell.", "watch it. Now you're using my lines. Poet has always been an easy target, I will agree. ;)", 'Because it will encourage teens to engage in riskier behavior. Abstinence until marriage is still the best way.', 'Obviously you missed the point. So sorry the the irony was beyond you.', "This is pure paranoia. What evidence do you have to support you're point of view? Why don't you actually do a little research yourself. Handgun Control Inc was founded by two victims of gun-violence. Sarah Brady joined Handgun Control Inc after her husband received brain damage from a bullet. Senator Ted Kennedy also supports gun-control because some of his family members were killed with guns. Many people who support gun control have experienced the horrors of gun violence firsthand and are just trying to make our nation a safer place. It's not because they are power hungry."]
['Therefore you accept that the Republican pa

In [0]:
import re
def clean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ",text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

Cleaning Data

In [20]:
quotes=[]
responses=[]
labels=[]
for sent1,sent2,l in zip(quote,response,label):
    quotes.append(clean(sent1))
    responses.append(clean(sent2))
    labels.append(dl.clean(str(l)))
print(labels[0:5])
print(quotes[0:5])
print(responses[0:5])

['1', '1', '1', '1', '1']
['First off That grade A USDA approved Liberalism in a nutshell ', 'watch it Now you are using my lines Poet has always been an easy target I will agree ; ', 'Because it will encourage teens to engage in riskier behavior Abstinence until marriage is still the best way ', 'Obviously you missed the point So sorry the the irony was beyond you ', 'This is pure paranoia What evidence do you have to support you are point of view Why do not you actually do a little research yourself Handgun Control Inc was founded by two victims of gun - violence Sarah Brady joined Handgun Control Inc after her husband received brain damage from a bullet Senator Ted Kennedy also supports gun - control because some of his family members were killed with guns Many people who support gun control have experienced the horrors of gun violence firsthand and are just trying to make our nation a safer place It not because they are power hungry ']
['Therefore you accept that the Republican par

In [21]:
labels=[int(i) for i in labels]
labels = keras.utils.to_categorical(labels)
print(labels[:5])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [0]:
f = open('glove.6B.50d.txt', encoding="utf8")
model = {}
for line in f:
    splitline = line.split()
    word = splitline[0]
    embedding = np.array([float(val) for val in splitline[1:]])
    model[word] = embedding

In [0]:
wordvec_model = model

In [26]:
data_inp_l, data_inp_r, embedding_matrix = dl.process_data(sent_l = quotes, sent_r = responses, wordVec_model = wordvec_model, dimx = 25, dimy = 25)

found  26250  unique words
number of unkown words:  10000
some unknown words  ['the', 'to', 'of', 'a', '$_START_$']


In [33]:
import sys
model = model_cnn2(dimx = 25, dimy = 25, embedding_matrix = embedding_matrix)
model.fit([data_inp_l,data_inp_r],labels,epochs=10,batch_size=8)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=3, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=3, kernel_size=2)`
  


Epoch 1/10
4692/4692 [==============================] - 5s 1ms/step - loss: 0.2525
Epoch 2/10
4692/4692 [==============================] - 4s 950us/step - loss: 0.2502
Epoch 3/10
4692/4692 [==============================] - 4s 948us/step - loss: 0.2502
Epoch 4/10
4692/4692 [==============================] - 4s 955us/step - loss: 0.2503
Epoch 5/10
3872/4692 [=======================>......] - ETA: 0s - loss: 0.2502

4692/4692 [==============================] - 4s 953us/step - loss: 0.2502
Epoch 6/10
4692/4692 [==============================] - 4s 959us/step - loss: 0.2502
Epoch 7/10
4692/4692 [==============================] - 4s 955us/step - loss: 0.2502
Epoch 8/10
4692/4692 [==============================] - 4s 951us/step - loss: 0.2503
Epoch 9/10
4692/4692 [==============================] - 4s 930us/step - loss: 0.2502
Epoch 10/10
 288/4692 [>.............................] - ETA: 4s - loss: 0.2497

4692/4692 [==============================] - 4s 949us/step - loss: 0.2502


TypeError: ignored